In [14]:
from glob import glob
import pandas as pd

modules = glob("../modules/*.csv")
modules
modules_df = []

for m in modules:
    temp_df = pd.read_csv(m)
    temp_df = temp_df.drop(columns=["Unnamed: 0"], errors="ignore")
    modules_df.append(temp_df)

full_model = pd.concat(modules_df)

dups = full_model.duplicated(subset="Attribute").sum()

model_path = "../../EL.data.model.csv"

if dups > 0:
    print(
        f"{dups} duplicates found in the data model. Check attributes to remove. All Attributes must be unique"
    )
else:
    print(f"Successfully joined modules. \n Creating data model at {model_path}")
    full_model.to_csv(model_path, index=False)

Successfully joined modules. 
 Creating data model at ../EL.data.model.csv


In [6]:
full_model.loc[full_model.duplicated(subset = 'Attribute', keep=False), ].sort_values('Attribute')

,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules,Module,Ontology,UsedIn,columnType


In [9]:
full_model.Module.unique()

array(['Instrument', 'Template', 'Analysis', 'ValidValues', 'Assay',
       'Other', 'Project', 'Ontology', 'Repository', 'SampleType',
       'Unspecified', 'Unit', 'Metadata', 'Model Organism',
       'BaseAnnotation'], dtype=object)

In [ ]:
# add new values
new_attrs_df = dm["DependsOn"].str.split(',').explode().dropna()
new_attrs_df = pd.DataFrame(new_attrs_df)
new_attrs_df['UsedIn'] = new_attrs_df.index

# use the old depends on as the new attributes
new_attrs_df = new_attrs_df.groupby("DependsOn").agg(lambda x: ",".join(np.unique(x)))
new_attrs_df.index.name = "Attribute"

print(new_attrs_df.index.is_unique)
new_attrs_df.head()

# remove existing values
new_attrs_df = new_attrs_df[~new_attrs_df.index.isin(dm.index)]
new_attrs_df.shape

In [ ]:
print('Supposed new shape:', np.array(new_attrs_df.shape) + np.array(dm.shape))

Supposed new shape [238  16]


In [ ]:
test = pd.concat([dm, new_attrs_df[~new_attrs_df.index.isin(dm.index)]])
print(test.shape)
test.head()

(238, 15)


,Description,Valid Values,DependsOn,DependsOn Component,Required,Parent,Validation Rules,Properties,Source,module,multivalue,columnType,Ontology,Notes,UsedIn
Attribute,,,,,,,,,,,,,,,
acquisitionBatchID,"Acquisition batch identifier, provided by the ...",NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
acquisitionBatchSize,The number of samples,NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
acquisitionBatchSizeUnit,The unit of measurement for number of samples ...,NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
acquisitionMode,The specific aspect of a mass spectrometer met...,NaN,NaN,NaN,True,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,https://www.ebi.ac.uk/ols/ontologies/ms/terms?...,A data contributor should be able to write in ...,Metabolomics
acquisitionSoftware,The name of the acquisition software used. Pro...,NaN,NaN,NaN,True,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,http://purl.obolibrary.org/obo/MS_1001455,A data contributor should be able to write in ...,Metabolomics


In [ ]:
test.index.is_unique

True

In [ ]:
test.to_csv("../../EL.data.model.csv")

In [ ]:
def join_data_model_partitions(partition_path):
    """Join the partitions back together to form the data model used in DCA

    Args:
        partition_path (str): directory containing the partitions as CSVs

    Returns:
        object: pandas dataframe
    """
    Parents = glob.glob(partition_path)
    df_dict = {pathlib.Path(m).stem: pd.read_csv(m) for m in Parents}
    data_model_full = (
        pd.concat([pd.read_csv(m) for m in Parents])
        .sort_values(by=["Parent", "Attribute"])
        .reset_index(drop=True)
        .fillna("")
    )

    data_model_full.info()

    return data_model

In [ ]:
# tests